In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import drive

In [4]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/chatbot-project')

Mounted at /content/drive


In [5]:
my_data = pd.read_csv('./preprocessed_qna_final.csv')
my_data.head(10)

,questions,answers
0,대성각의 대표적인 메뉴를 알 수 있나요?,"대성각의 메뉴에는 짜장면, 짬뽕이 있습니다"
1,대성각의 영업시간은 어떻게 되나요?,대성각의 영업시간은 11:00 - 15:00/21:00입니다
2,대성각의 연락처를 알 수 있나요?,대성각의 연락처는 02-356-2194입니다
3,대성각에 인접한 시설을 알 수 있나요?,"대성각에 인접한 시설에는 역촌역 3번출구, 그림나라아동미술, 메리피아노음악교습소가 ..."
4,대성각의 휴무일을 알 수 있나요?,대성각의 휴무일은 매주 일요일입니다
5,대성각의 위치를 알 수 있나요?,대성각의 위치는 서울 은평구 녹번로 7입니다
6,대성각의 주차시설이 있나요?,대성각의 주차시설은 없습니다
7,대성각의 주차시설이 있는지 알 수 있나요?,대성각의 주차시설은 없습니다
8,대성각에 인접한 시설이 있나요?,"대성각에 인접한 시설에는 역촌역 3번출구, 그림나라아동미술, 메리피아노음악교습소가 ..."
9,대성각의 주요 메뉴를 알 수 있나요?,"대성각의 메뉴에는 짜장면, 짬뽕이 있습니다"


In [6]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-b3dxkno6/kobert-tokenizer_43745fec09ed4251b20b012b79082219
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-b3dxkno6/kobert-tokenizer_43745fec09ed4251b20b012b79082219
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=5ad1af24f63f19430bd6fac174740feb46554883b899511cca5ec3382308b2a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ltg_9r0_/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [7]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("대성각에 인접한 시설이 있나요?")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2,
 1633,
 6573,
 5336,
 6896,
 3758,
 7225,
 7828,
 2981,
 7096,
 3854,
 5655,
 6999,
 258,
 3]

In [8]:
questions = my_data['questions']
answers = my_data['answers']

# # 토큰화
question_tokens = [tokenizer.encode(q, add_special_tokens=True) for q in questions]
answer_tokens = [tokenizer.encode(a, add_special_tokens=True) for a in answers]
question_tokens[:5], answer_tokens[:5]

([[2,
   1633,
   6573,
   5336,
   7095,
   1678,
   2017,
   6116,
   3166,
   2872,
   3854,
   5655,
   6999,
   258,
   3],
  [2,
   1633,
   6573,
   5336,
   7095,
   3382,
   6706,
   7086,
   3225,
   1763,
   5655,
   6999,
   258,
   3],
  [2,
   1633,
   6573,
   5336,
   7095,
   3343,
   7416,
   6116,
   3166,
   2872,
   3854,
   5655,
   6999,
   258,
   3],
  [2,
   1633,
   6573,
   5336,
   6896,
   3758,
   7225,
   7828,
   2981,
   7088,
   3166,
   2872,
   3854,
   5655,
   6999,
   258,
   3],
  [2,
   1633,
   6573,
   5336,
   7095,
   5191,
   6228,
   7126,
   7088,
   3166,
   2872,
   3854,
   5655,
   6999,
   258,
   3]],
 [[2,
   1633,
   6573,
   5336,
   7095,
   2017,
   6900,
   4396,
   7178,
   6198,
   46,
   517,
   7367,
   6481,
   7096,
   3867,
   3],
  [2,
   1633,
   6573,
   5336,
   7095,
   3382,
   6706,
   7086,
   538,
   251,
   524,
   543,
   251,
   59,
   133,
   251,
   7139,
   3],
  [2,
   1633,
   6573,
   5336,
   7095,
 

In [9]:
cls_token_id = tokenizer.cls_token_id
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
print(cls_token_id, sep_token_id, pad_token_id)

2 3 1


In [10]:
# 토큰화된 리스트를 PyTorch 텐서로 변환
questions_tensors = [torch.tensor(t, dtype=torch.long) for t in question_tokens]
answers_tensors = [torch.tensor(t, dtype=torch.long) for t in answer_tokens]
questions_tensors[:5], answers_tensors[:5]

([tensor([   2, 1633, 6573, 5336, 7095, 1678, 2017, 6116, 3166, 2872, 3854, 5655,
          6999,  258,    3]),
  tensor([   2, 1633, 6573, 5336, 7095, 3382, 6706, 7086, 3225, 1763, 5655, 6999,
           258,    3]),
  tensor([   2, 1633, 6573, 5336, 7095, 3343, 7416, 6116, 3166, 2872, 3854, 5655,
          6999,  258,    3]),
  tensor([   2, 1633, 6573, 5336, 6896, 3758, 7225, 7828, 2981, 7088, 3166, 2872,
          3854, 5655, 6999,  258,    3]),
  tensor([   2, 1633, 6573, 5336, 7095, 5191, 6228, 7126, 7088, 3166, 2872, 3854,
          5655, 6999,  258,    3])],
 [tensor([   2, 1633, 6573, 5336, 7095, 2017, 6900, 4396, 7178, 6198,   46,  517,
          7367, 6481, 7096, 3867,    3]),
  tensor([   2, 1633, 6573, 5336, 7095, 3382, 6706, 7086,  538,  251,  524,  543,
           251,   59,  133,  251, 7139,    3]),
  tensor([   2, 1633, 6573, 5336, 7095, 3343, 7416, 5760,  517,   84,  142,  184,
            47,  133,  243, 7139,    3]),
  tensor([   2, 1633, 6573, 5336, 6896, 3758, 722

In [11]:
# 패딩을 위한 최대 길이 결정
questions_max_length = max(tensor.size(0) for tensor in questions_tensors)
answers_max_length = max(tensor.size(0) for tensor in answers_tensors)
max_length = max(questions_max_length, answers_max_length)
max_length

71

In [12]:
# 패딩 함수
def pad_or_truncate(tensors, max_length, pad_token_id):
    padded_tensors = []
    for tensor in tensors:
        if tensor.size(0) < max_length:
            padded_tensor = torch.cat([tensor, torch.tensor([pad_token_id] * (max_length - tensor.size(0)), dtype=torch.long)])
        else:
            padded_tensor = tensor[:max_length]
        padded_tensors.append(padded_tensor)
    return pad_sequence(padded_tensors, batch_first=True)

# 패딩 및 트렁케이팅 적용
questions_padded = pad_or_truncate(questions_tensors, max_length, pad_token_id=pad_token_id)
answers_padded = pad_or_truncate(answers_tensors, max_length, pad_token_id=pad_token_id)

# 패딩된 텐서의 모양
print("Padded Questions Shape:", questions_padded.shape)
print("Padded Answers Shape:", answers_padded.shape)
print("Padded Questions head:", questions_padded[:5])
print("Padded Answers head:", answers_padded[:5])

Padded Questions Shape: torch.Size([26691, 71])
Padded Answers Shape: torch.Size([26691, 71])
Padded Questions head: tensor([[   2, 1633, 6573, 5336, 7095, 1678, 2017, 6116, 3166, 2872, 3854, 5655,
         6999,  258,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 1633, 6573, 5336, 7095, 3382, 6706, 7086, 3225, 1763, 5655, 6999,
          258,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    

In [105]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=max_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [106]:
class TransformerModel(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, vocab_size, max_len=max_length):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          batch_first=True)
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)

        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)

        output = self.transformer(src, tgt,
                                  tgt_mask=tgt_mask,
                                  src_key_padding_mask=src_key_padding_mask,
                                  tgt_key_padding_mask=tgt_key_padding_mask)
        return self.fc_out(output)

In [107]:
class EarlyStopping:
    def __init__(self, patience=4, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = float('inf')
        self.counter = 0
        self.stopped_early = False

    def __call__(self, train_loss, model):
        if train_loss < self.best_loss:
            self.best_loss = train_loss
            self.counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            self.counter += 1
            if self.verbose:
                print(f'Training loss did not improve. Counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.stopped_early = True
                if self.verbose:
                    print('Early stopping triggered.')

In [108]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [104]:
vocab_size = tokenizer.vocab_size
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048

In [110]:
# 데이터셋 생성
dataset = TensorDataset(questions_padded, answers_padded)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=2, pin_memory=True)

# 모델, 옵티마이저 생성
model = TransformerModel(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                         num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                         vocab_size=vocab_size, max_len=max_length)

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)

early_stopping = EarlyStopping(patience=4, verbose=True)

scaler = GradScaler()

# 학습
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch in dataloader:
        src_batch, tgt_batch = batch

        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        src_key_padding_mask = (src_batch == pad_token_id).to(device)
        tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

        optimizer.zero_grad()

        with autocast():
            output = model(src_batch, tgt_input,
                       src_key_padding_mask=src_key_padding_mask,
                       tgt_key_padding_mask=tgt_key_padding_mask)
            loss = criterion(output.contiguous().view(-1, vocab_size), tgt_output.contiguous().view(-1))

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.6f}")

    early_stopping(avg_loss, model)
    if early_stopping.stopped_early:
        break

print("Training complete!")

Epoch 1/50, Train Loss: 5.838479
Epoch 2/50, Train Loss: 3.461761
Epoch 3/50, Train Loss: 2.546593
Epoch 4/50, Train Loss: 2.078056
Epoch 5/50, Train Loss: 1.715536
Epoch 6/50, Train Loss: 1.421165
Epoch 7/50, Train Loss: 1.185781
Epoch 8/50, Train Loss: 1.004832
Epoch 9/50, Train Loss: 0.873679
Epoch 10/50, Train Loss: 0.766108
Epoch 11/50, Train Loss: 0.678648
Epoch 12/50, Train Loss: 0.601871
Epoch 13/50, Train Loss: 0.531182
Epoch 14/50, Train Loss: 0.467397
Epoch 15/50, Train Loss: 0.412278
Epoch 16/50, Train Loss: 0.360238
Epoch 17/50, Train Loss: 0.316889
Epoch 18/50, Train Loss: 0.277136
Epoch 19/50, Train Loss: 0.243517
Epoch 20/50, Train Loss: 0.187528
Epoch 21/50, Train Loss: 0.156901
Epoch 22/50, Train Loss: 0.142929
Epoch 23/50, Train Loss: 0.131215
Epoch 24/50, Train Loss: 0.123601
Epoch 25/50, Train Loss: 0.116124
Epoch 26/50, Train Loss: 0.110995
Epoch 27/50, Train Loss: 0.106409
Epoch 28/50, Train Loss: 0.101853
Epoch 29/50, Train Loss: 0.098365
Epoch 30/50, Train Loss

In [114]:
def generate_text(model, src, tokenizer, max_length=max_length, device='cuda'):
    src = src.to(device)
    src_key_padding_mask = (src == pad_token_id).to(device)

    with torch.no_grad():
        tgt_input = torch.tensor([[cls_token_id]], device=device, dtype=torch.long)
        for _ in range(max_length):
            tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

            output = model(src, tgt_input, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask)
            output_probs = F.softmax(output[:, -1, :], dim=-1)
            next_token = torch.argmax(output_probs, dim=-1)

            tgt_input = torch.cat((tgt_input, next_token.unsqueeze(0)), dim=1)

            if next_token.item() == sep_token_id:
                break

    return tgt_input[:, 1:].squeeze(0)  # Remove CLS token

In [115]:
def decode_output(output_tokens, tokenizer):
    tokens = tokenizer.convert_ids_to_tokens(output_tokens)
    return tokenizer.convert_tokens_to_string([token for token in tokens if token not in [tokenizer.pad_token, tokenizer.sep_token]])

In [116]:
# 평가
model.eval()
generated_texts = []
target_texts = []

try:
    with torch.no_grad():
        # 첫 번째 배치만 가져옵니다
        batch = next(iter(dataloader))
        src_batch, tgt_batch = batch
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        for i, (src_single, tgt_single) in enumerate(zip(src_batch, tgt_batch)):
            # Beam Search를 사용하여 텍스트 생성
            generated = generate_text(model, src_single.unsqueeze(0), tokenizer, max_length=max_length, device=device)
            input_text = tokenizer.decode(src_single.cpu().numpy(), skip_special_tokens=True)
            predicted_text = decode_output(generated.cpu().numpy(), tokenizer)  # 이 부분에서 차원 확인
            target_text = tokenizer.decode(tgt_single.cpu().numpy(), skip_special_tokens=True)

            print(f"\nSample {i+1}:")
            print(f"Input: {input_text}")
            print(f"Predicted: {predicted_text}")
            print(f"Target: {target_text}")

            generated_texts.append(predicted_text)
            target_texts.append(target_text)

            if i == 9:
                break

except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

print("Evaluation complete!")


Sample 1:
Input: 창성해물손칼국수의 대표적인 메뉴를 알 수 있나요?
Predicted: 창성해물손칼국수의 메뉴에는 해물칼국수, 해물수제비가 있습니다
Target: 창성해물손칼국수의 메뉴에는 해물칼국수, 해물수제비가 있습니다

Sample 2:
Input: 족팔계의 영업시간은 어떻게 되나요?
Predicted: 도깨비굴의 영업시간은 11:00 - 21:00입니다
Target: 족팔계의 영업시간은 16:00 - 01:30입니다

Sample 3:
Input: 정화네동태국의 주차시설이 있는지 알 수 있나요?
Predicted: 정화네동태국의 위치는 서울 동대문구 왕산로 254 1층입니다
Target: 정화네동태국의 주차시설은 있습니다

Sample 4:
Input: 플랫커피의 주차시설이 있는지 알 수 있나요?
Predicted: 도화루중화요리의 메뉴에는 홍어회, 떡볶이가 있습니다
Target: 플랫커피의 주차시설은 없습니다

Sample 5:
Input: 명륜진사갈비 신정네거리점의 위치를 알 수 있나요?
Predicted: 명륜진사갈비 신정네거리점의 위치는 서울 양천구 중앙로 222입니다
Target: 명륜진사갈비 신정네거리점의 위치는 서울 양천구 중앙로 222입니다

Sample 6:
Input: 논밭골 봉천점의 영업시간은 어떻게 되나요?
Predicted: 논밭골 봉천점의 위치는 서울 관악구 청룡길 30입니다
Target: 논밭골 봉천점의 영업시간은 10:40 - 24:00입니다

Sample 7:
Input: 영등포구 맛집 추천좀
Predicted: 영등포구의 맛집으로는 도도포차에 가보세요
Target: 영등포구의 맛집으로는 꽃담식당를 추천드립니다

Sample 8:
Input: 은평구 맛집 추천좀
Predicted: 은평구의 맛집으로는 도화에 가보세요
Target: 은평구의 맛집으로는 연탄 생고기집 대조점이 알려져 있습니다

Sample 9:
Input: 아날로그소사이어티키친의 연락처가 어떻게 되나요?
Predicted: 아날로그소사이어티키친의

In [119]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 학습
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch in dataloader:
        src_batch, tgt_batch = batch

        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        tgt_input = tgt_batch[:, :-1]
        tgt_output = tgt_batch[:, 1:]

        src_key_padding_mask = (src_batch == pad_token_id).to(device)
        tgt_key_padding_mask = (tgt_input == pad_token_id).to(device)

        optimizer.zero_grad()

        with autocast():
            output = model(src_batch, tgt_input,
                       src_key_padding_mask=src_key_padding_mask,
                       tgt_key_padding_mask=tgt_key_padding_mask)
            loss = criterion(output.contiguous().view(-1, vocab_size), tgt_output.contiguous().view(-1))

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.6f}")

    early_stopping(avg_loss, model)
    if early_stopping.stopped_early:
        break

print("Training complete!")

Epoch 1/20, Train Loss: 0.057340
Epoch 2/20, Train Loss: 0.055851
Epoch 3/20, Train Loss: 0.055248
Epoch 4/20, Train Loss: 0.054614
Epoch 5/20, Train Loss: 0.050829
Epoch 6/20, Train Loss: 0.050559
Epoch 7/20, Train Loss: 0.049952
Epoch 8/20, Train Loss: 0.049694
Epoch 9/20, Train Loss: 0.049510
Epoch 10/20, Train Loss: 0.049437
Epoch 11/20, Train Loss: 0.049109
Epoch 12/20, Train Loss: 0.048997
Epoch 13/20, Train Loss: 0.048719
Epoch 14/20, Train Loss: 0.047074
Epoch 15/20, Train Loss: 0.046247
Epoch 16/20, Train Loss: 0.046245
Epoch 17/20, Train Loss: 0.045991
Epoch 18/20, Train Loss: 0.045848
Epoch 19/20, Train Loss: 0.045529
Epoch 20/20, Train Loss: 0.045479
Training complete!


In [120]:
# 평가
model.eval()
generated_texts = []
target_texts = []

try:
    with torch.no_grad():
        # 첫 번째 배치만 가져옵니다
        batch = next(iter(dataloader))
        src_batch, tgt_batch = batch
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        for i, (src_single, tgt_single) in enumerate(zip(src_batch, tgt_batch)):
            generated = generate_text(model, src_single.unsqueeze(0), tokenizer, max_length=100, device=device)

            input_text = tokenizer.decode(src_single.cpu().numpy(), skip_special_tokens=True)
            predicted_text = decode_output(generated.cpu().numpy(), tokenizer)
            target_text = tokenizer.decode(tgt_single.cpu().numpy(), skip_special_tokens=True)

            print(f"\nSample {i+1}:")
            print(f"Input: {input_text}")
            print(f"Predicted: {predicted_text}")
            print(f"Target: {target_text}")

            generated_texts.append(predicted_text)
            target_texts.append(target_text)

            if i == 9:
                break

except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

print("Evaluation complete!")


Sample 1:
Input: 할매떡볶이에 인접한 시설이 있나요?
Predicted: 할매떡볶이에 인접한 시설에는 사직역 3번 출구, 농협하나로클럽, 스타벅스가 있습니다
Target: 할매떡볶이에 인접한 시설에는 사직역 3번 출구, 농협하나로클럽, 스타벅스가 있습니다

Sample 2:
Input: 깐부치킨 뚝섬유원지점의 메뉴를 알 수 있나요?
Predicted: 깐부치킨 선유도역점의 메뉴에는 크런치 찰 핫도그, 빠삭커리네윙봉이 있습니다
Target: 깐부치킨 뚝섬유원지점의 메뉴에는 깐부 해물짬뽕탕, 크런치 찰 핫도그가 있습니다

Sample 3:
Input: 소담촌 용두점의 휴무일이 어떻게 되나요?
Predicted: 소담촌 아차산역점의 휴무일은 없습니다
Target: 소담촌 용두점의 휴무일은 없습니다

Sample 4:
Input: 태양부양꼬치 시흥점의 위치를 알 수 있나요?
Predicted: 태양부양꼬치 시흥점의 위치는 서울 금천구 은행나무로 51입니다
Target: 태양부양꼬치 시흥점의 위치는 서울 금천구 은행나무로 51입니다

Sample 5:
Input: 히치케이커의 대표적인 메뉴는 무엇이 있나요?
Predicted: 히포슈가 송파본점의 메뉴에는 플레인 팬케이크, 루꼴라 리코타 샐러드 팬케이크가 있습니다
Target: 히치케이커의 메뉴에는 아이스아메리카노, 바닐라라떼가 있습니다

Sample 6:
Input: 성동구 맛집 추천 부탁해
Predicted: 성동구의 맛집으로는 에롤파가 있습니다
Target: 성동구의 맛집으로는 성수피글렛를 추천드립니다

Sample 7:
Input: 목탄장 여의도점의 주차시설이 있는지 알 수 있나요?
Predicted: 목탄장 여의도점의 메뉴에는 구운 포카치아, 과일 단새우가 있습니다
Target: 목탄장 여의도점의 주차시설은 있습니다

Sample 8:
Input: 바르다김선생 당산역점에 인접한 시설이 있나요?
Predicted: 바르다김선생 당산역점에 인접한 시설에는 당산역 12번 출구, 메가커피, 맘스터치가 있

In [121]:
# 메인 루프
def main(model, tokenizer, device):
    while True:
        question = input("질문을 입력하세요 (종료하려면 '나가기' 입력): ")
        if question.strip() == '나가기':
            print("프로그램을 종료합니다.")
            break

        # 질문을 토크나이즈
        input_ids = tokenizer(question, return_tensors="pt", padding=True, truncation=True).input_ids

        # 응답 생성
        response_ids = generate_text(model, input_ids, tokenizer, device=device)

        # 응답 디코딩
        response = tokenizer.decode(response_ids, skip_special_tokens=True)
        print(f"답변: {response}\n")

main(model, tokenizer, device)

질문을 입력하세요 (종료하려면 '나가기' 입력): 덮세권의 대표적인 메뉴는 무엇이 있나요?
답변: 덮세권의 메뉴에는 부타동, 규동가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 성동구 맛집 추천 부탁해
답변: 성동구의 맛집으로는 히어를 추천드립니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 히치케이커의 대표적인 메뉴는 무엇이 있나요?
답변: 히치케이커의 메뉴에는 아이스아메리카노, 바닐라라떼가 있습니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 태양부양꼬치 시흥점의 위치를 알 수 있나요?
답변: 태양부양꼬치 시흥점의 위치는 서울 금천구 은행나무로 51입니다

질문을 입력하세요 (종료하려면 '나가기' 입력): 나가기
프로그램을 종료합니다.
